In [1]:
import gensim
import os
import re
import pickle
import numpy as np
from gensim.scripts.glove2word2vec import glove2word2vec

In [5]:
## Input
#
# ID    dataset        Support_Year
# 0     GoogleNews     -
# 1     NYT            3yr
# 2     Common Crawl   -
# 3     Google Books   10yr
# 4     Twitter        -
data_id = 1
word1 = 'china'
word2 = 'canada'
year = 1987

In [41]:
def choose_data(data_id,year):
    dataset = ['GoogleNews','NYTw2v','CommonCrawlw2v','COHAw2v','Twitterw2v']
    dir_loc = "../data/Dataset/"+dataset[data_id]+"/"
    subdir = os.listdir(dir_loc)
    if(".DS_Store" in subdir):
        subdir.remove(".DS_Store") 
    if(len(subdir)==1):
        print(dataset[data_id],"does not support year search")
        file_name = dir_loc+subdir[0]
    else:
        print(dataset[data_id],"support year search")
        file_name = None
        for file in subdir:
            m = re.search('\d{4}.\d{4}', file)
            if(m):
                year_range = m.group(0)
                if(year and year >= int(year_range[:4]) and year <= int(year_range[-4:])):
                    file_name = dir_loc+file
                    break
    if(file_name):     
        try:
            w2v=gensim.models.KeyedVectors.load_word2vec_format(file_name,binary=False,unicode_errors='ignore')
        except:
            w2v=gensim.models.KeyedVectors.load_word2vec_format(file_name,binary=True,unicode_errors='ignore')
        return w2v
    else:
        print("can not find corresponding dataset, nothing returned")
        return None

def similarity(word1,word2,w2v):
    try:
        print(word1,word2,":",w2v.similarity(word1,word2))
    except Exception as e:
        print(str(e))

In [42]:
# Google News
w2v_googlenews = choose_data(0,1987)

GoogleNews does not support year search


In [43]:
# word2vec distinguish uppercase and lowercase
similarity(word1,word2,w2v_googlenews)

china canada : 0.28617182


/Users/sunny/anaconda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [44]:
# NYT
# glove only allow lowercase
w2v_nyt = choose_data(1,1991)

NYTw2v support year search


In [45]:
similarity('china','canada',w2v_nyt)

china canada : 0.2935132


/Users/sunny/anaconda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [18]:
# Common Crawl
w2v_commoncrawl = choose_data(2,1987)
similarity('china','canada',w2v_commoncrawl)

CommonCrawlw2v does not support year search
../data/Dataset/CommonCrawlw2v/glove.42B.300d.txt
"word 'china' not in vocabulary"


In [239]:
# COHA
w2v_coha = choose_data(3,1987)
similarity('china','canada',w2v_coha)

COHAw2v support year search
china canada : 0.36149564


/Users/sunny/anaconda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [15]:
# Twitter
w2v_twitter = choose_data(4,1987)
similarity('china','canada',w2v_twitter)

Twitterw2v does not support year search
china canada : 0.52033085


/Users/sunny/anaconda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# Transfer from glove to w2v

In [250]:
# transfer CommanCrawl to word2vec format
input_dir = '../data/Dataset/CommonCrawl/'
output_dir = '../data/Dataset/CommonCrawlw2v/'
for file in os.listdir(input_dir):
    glove2word2vec(input_dir+file, output_dir+file)

In [8]:
# transfer NYT to word2vec format
input_dir = '../data/Dataset/NYT/'
output_dir = '../data/Dataset/NYTw2v/'
for file in os.listdir(input_dir):
    glove2word2vec(input_dir+file, output_dir+file)

In [12]:
# transfer Twitter to word2vec format
input_dir = '../data/Dataset/Twitter/'
output_dir = '../data/Dataset/Twitterw2v/'
for file in os.listdir(input_dir):
    glove2word2vec(input_dir+file, output_dir+file)

# Transfer from Dynamic to w2v

In [231]:
# transfer COHA to word2vec format and rename as year-year
for i in range(1810,2010,10):
    dir_loc = '../data/Dataset/COHA/sgns/'
    pkl_name = str(i)+'-vocab.pkl'
    npy_name = str(i)+'-w.npy'
    output_loc = '../data/Dataset/COHAw2v/'+str(i)+'-'+str(i+9)
    with open(dir_loc+pkl_name, 'rb') as f:
        vocab = pickle.load(f)
    vector = np.load(dir_loc+npy_name)
    data =np.concatenate((np.array([vocab]).T,vector),axis = 1)
    with open(output_loc,'w') as f:
        for j in range(len(data)):
            if(j == 0):
                f.write(str(len(vocab))+" "+str(len(vector[0]))+'\n')
            f.write(" ".join(list(data[j]))+'\n')

# Questions

1. years in Twitter overlap
2. Google News distinguish Uppercase and Lowercase